![bayt logo](https://images.g2crowd.com/uploads/product/image/social_landscape/social_landscape_b9320393d0092c1c2f9f5e1d78197b14/bayt-com.png)

# Bayt website links scrapping
**Note: All the links were collected on 20-12-2022**

In this notebook we will scrape all the job links in bayt website

In [ ]:
# import all the needed libraries
import requests
from bs4 import BeautifulSoup
import csv
import numpy as np
import threading

### Get all the jobs pages from the site.
#### The number 304 is number of pages in the site you can go to this [link](https://www.bayt.com/en/egypt/jobs/?page=1) and get the last page number and put it instead of 304 


In [ ]:
urls = [f'https://www.bayt.com/en/egypt/jobs/?page={i}' for i in range(1, 304)]

In [ ]:
def get_links(urls, thread_number):
    # create or open new csv file to save all the links
    csv_file = open(f'bayt_job_links_20-12-2022_{thread_number}.csv', 'a+',encoding='utf-16')
    writer = csv.DictWriter(csv_file, fieldnames=['title', 'link'])
    writer.writeheader()
    for url_index, url in enumerate(urls):
        # make a request to the page to get the data to be scraped
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)
        
        # convert the response to soup object to be able to extract information form it
        soup = BeautifulSoup(response.content, 'html.parser')
        jobs = soup.find_all('li', class_='has-pointer-d')
        print(f'Thread #{thread_number}: start url: {url_index}')
        
        # start the link and title extraction form the page
        for job_index, job in enumerate(jobs):
            print(f'Thread #{thread_number}: Job: {job_index}')
            title = job.find('h2', class_='jb-title').text.strip()
            link = job.find('h2', class_='jb-title').a.get('href')
            
            # add the link to the csv file
            writer.writerow({
              'link': f'https://www.bayt.com{link}',
              'title': title
            })
    print('-' * 50)

### As you can see there are alot of pages that I need to scrape and it could even get bigger so for ease of use and time I have used mutithreads to split the data 

In [ ]:
splitied_urls = np.array_split(urls, 8) # I have used array_split to split my data to n (8) numbers of smaller lists you can change the 8 number to be number of threads you want to use
threads = []
# start the thread for each list in my case 8 lists so 8 threads
for index, mini_urls in enumerate(splitied_urls):
    thread = threading.Thread(target=get_links, kwargs={'urls': mini_urls, 'thread_number': str(index + 1)})
    thread.start()
    threads.append(thread)

# after ending the application re join the threads again (like closing the threads)
for thread in threads:
    thread.join()